In [1]:
import numpy as np
import osmnx as ox
import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt

import random
import pickle

import mlrfit as mf
import lrrouting as ldr

/Users/parshakova.tanya/opt/anaconda3/envs/routing/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rank = 35
n = 1000

G = nx.fast_gnp_random_graph(n, p=0.05, directed=True)
Adj, Dist, nodes_cc = ldr.nx_graph_to_matrices(G, nodes=True)
adjacency_list = ldr.adjacency_directed_list(Adj)
diam_G = Dist.max()
w_min = Dist[Dist>0].min()
m = n = len(nodes_cc)
print(f"{n=}, {G.number_of_nodes()=}, {G.number_of_edges()=}")


place = "Pacifica, CA, USA"
G, Adj, Dist, nodes_cc = ldr.dist_matrix_osmnx(place, directed=True, nodes=True)
m = n = Dist.shape[0]
diam_G = Dist.max()
w_min = Dist[Dist>0].min()
print(m, diam_G, w_min, (Adj>0).sum())

[1000]
n_cc = 1000, n0 = 1000
in  degrees: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 1, 30: 1, 31: 2, 32: 1, 33: 4, 34: 5, 35: 0, 36: 4, 37: 11, 38: 16, 39: 16, 40: 18, 41: 15, 42: 28, 43: 30, 44: 35, 45: 51, 46: 64, 47: 51, 48: 59, 49: 62, 50: 41, 51: 66, 52: 52, 53: 57, 54: 48, 55: 34, 56: 39, 57: 49, 58: 26, 59: 24, 60: 17, 61: 13, 62: 8, 63: 13, 64: 9, 65: 9, 66: 8, 67: 3, 68: 3, 69: 2, 70: 2, 71: 0, 72: 2}
out degrees: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 2, 32: 3, 33: 3, 34: 2, 35: 6, 36: 9, 37: 11, 38: 15, 39: 19, 40: 14, 41: 22, 42: 26, 43: 32, 44: 38, 45: 38, 46: 54, 47: 51, 48: 63, 49: 48, 50: 60, 51: 58, 52: 54, 53: 43, 54: 49, 55: 40, 56: 43, 57: 40

In [3]:
rt_max_iters = min(int(5*Dist.max()/w_min), (10**4) // 2)
adjacency_list = ldr.adjacency_directed_list(Adj)
sources, targets = ldr.st_pairs(n, Dist, 5020)
M = min(5000, sources.size)
sources = sources[:M]
targets = targets[:M]

PSD = False
symm = np.allclose(Dist, Dist.T) if m==n else False

hpart = mf.random_hpartition(m, n, num_levels=1, symm=symm, perm=False)
B1, C1 = mf.single_level_factor_fit(Dist, np.array([rank]), hpart, level=0, symm=symm, PSD=PSD)[:2]

lr_dar = ldr.construct_lr_graph(B1, C1, adjacency_list)
lr_A = B1 @ C1.T

In [4]:
assert np.allclose(ldr.adjacency_list_to_matrix(adjacency_list), Adj)
print("PASSED adjacency_list_to_matrix")

PASSED adjacency_list_to_matrix


In [5]:
def route_st_dar(s, t, dar, del_cycle=False, max_iters=50):
    route = [s]
    a = s; wpath = 0.
    if del_cycle: 
        route, _ = dar.route_with_dfs(s=s, t=t)
    else:
        while len(route) < max_iters and a != t:
            if del_cycle:
                b_next, w_b_next = dar.next_node_del_cycles(s=s, t=t, a=a)
            else:
                b_next, w_b_next = dar.next_node(s=s, t=t, a=a)
            wpath += w_b_next
            a = b_next
            route += [b_next]
            if len(dar.adjacency_list[a]) == 0:
                break
    return route


def route_st_matrix(s, t, func_next_node, adjacency_list):
    route = [s]
    max_iters = 50
    a = s; wpath = 0.
    while len(route) < max_iters and a != t:
        b_next = func_next_node(s=s, t=t, a=a)
        for b, w_ab in adjacency_list[a]:
            if b == b_next: 
                w_a_b_next = w_ab 
                break
        wpath += w_a_b_next
        a = b_next
        route += [b_next]
        if len(adjacency_list[a]) == 0:
            break
    return route

In [6]:
total_matches = 0
dist_func = lambda b,t: lr_A[b,t]
func_next_node = ldr.next_node(dist_func, adjacency_list, Adj)

for (s, t) in zip(sources, targets):
    if s == t: continue
    r1 = route_st_dar(s, t, lr_dar, del_cycle=False)
    r2 = route_st_matrix(s, t, func_next_node, adjacency_list)
    total_matches += np.allclose(r1, r2) if len(r1)==len(r2) else 0
    for a in r1[1:]:
        d_at1, d_at2 = np.zeros(len(adjacency_list[a])), np.zeros(len(adjacency_list[a]))
        for idx, (b, w_b) in enumerate(adjacency_list[a]):
            d_at1[idx] = w_b + lr_dar.dist(b, t)
            d_at2[idx] = Adj[a, b] + dist_func(b, t)
        assert np.allclose(d_at2, d_at1)
        b_next1 = lr_dar.next_node(s=s, t=t, a=a)[0]
        b_next2 = func_next_node(s=s, t=t, a=a)
        # print(f"{adjacency_list[a]} \n{d_at1=} {np.argsort(d_at1)} \n{d_at2=} {np.argsort(d_at2)}\n")
        assert b_next1 == adjacency_list[a][np.argmin(d_at1)][0] \
                and b_next2 == adjacency_list[a][np.argmin(d_at2)][0]
print("PASSED, fraction of fully matched paths is %.4f"%( total_matches / len(sources)))

PASSED, fraction of fully matched paths is 0.9990


In [7]:
import cvxpy as cp

In [8]:
x = cp.Variable(10)
f = cp.square(cp.norm_inf(x))
f.curvature

'CONVEX'

In [9]:
["foo", "bar", "baz"].index("bar")

1